In [389]:
import numpy as np
import pandas as pd
import string
from tqdm import tqdm
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import openai

In [390]:
with open('stopwords.txt') as f:
    stopwords = f.read().replace('\n',' ').split()

In [391]:
with open('training_text.txt', encoding='utf-8') as f:
    text = f.read().replace('\n','')
    print(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([t for t in text if t not in list('0123456789')])
    text = text.replace('”', '').replace('“', '').replace('’', '').lower().split()

text = [w for w in text if w not in stopwords][:2000]

Analyze the following text for words similar to masculine and feminine : The study of gender and ethnic stereotypes is an importanttopic across many disciplines. Language analysis is a standardtool used to discover, understand, and demonstrate such stereotypes(1–5). Previous literature broadly establishes that languageboth reflects and perpetuates cultural stereotypes. However,such studies primarily leverage human surveys (6–16), dictionaryand qualitative analysis (17), or in-depth knowledge of differentlanguages (18). These methods often require time-consumingand expensive manual analysis and may not easily scale acrosstypes of stereotypes, time periods, and languages. In this paper,we propose using word embeddings, a commonly used tool innatural language processing (NLP) and machine learning, as aframework to measure, quantify, and compare beliefs over time.As a specific case study, we apply this tool to study the temporaldynamics of gender and ethnic stereotypes in the 20th and 21st

In [392]:
WINDOW_SIZE = 3
NUM_NEGATIVE_SAMPLES = 3

data = []

#iterate over all words
for idx,center_word in enumerate(text[WINDOW_SIZE-1:-WINDOW_SIZE]):
    
    #iterate over the context words around the center word
    context_words = [context_word for context_word in text[idx:idx+2*WINDOW_SIZE-1] if context_word != center_word]
    for context_word in context_words:
        
        #get words NOT in the current context as negative examples
        data.append([center_word, context_word, 1])
        negative_samples = np.random.choice([w for w in text[WINDOW_SIZE-1:-WINDOW_SIZE] if w != center_word and w not in context_words], NUM_NEGATIVE_SAMPLES)
        
        for negative_samp in negative_samples:
            
            #add a training row
            data.append([center_word, negative_samp, 0])

In [393]:
df = pd.DataFrame(columns=['center_word', 'context_word', 'label'], data=data)
words = np.intersect1d(df.context_word, df.center_word)
df = df[(df.center_word.isin(words)) & (df.context_word.isin(words))].reset_index(drop=True)

In [394]:
def sigmoid(v, scale=1):
    return 1 / (1 + np.exp(-scale*v))

In [395]:
def update_embeddings(df, main_embeddings, context_embeddings, learning_rate, debug=False):
    
    #get differences between main embeddings and corresponding context embeddings
    main_embeddings_center = main_embeddings.loc[df.center_word].values
    context_embeddings_context = context_embeddings.loc[df.context_word].values
    diffs = context_embeddings_context - main_embeddings_center
    
    #get similarities, scores, and errors between main embeddings and corresponding context embeddings
    dot_prods = np.sum(main_embeddings_center * context_embeddings_context, axis=1)
    scores = sigmoid(dot_prods)
    errors = (df.label - scores).values.reshape(-1,1)
    
    #calculate updates
    updates = diffs*errors*learning_rate
    updates_df = pd.DataFrame(data=updates)
    updates_df['center_word'] = df.center_word
    updates_df['context_word'] = df.context_word
    updates_df_center = updates_df.groupby('center_word').sum()
    updates_df_context = updates_df.groupby('context_word').sum()
    
    if debug:
        plot_words(debug)
    
    #apply updates
    main_embeddings += updates_df_center.loc[main_embeddings.index]
    context_embeddings -= updates_df_context.loc[context_embeddings.index]
    
    #normalize embeddings
    main_embeddings = normalize_data(main_embeddings)
    context_embeddings = normalize_data(context_embeddings)
    
    #return the updated embeddings
    return main_embeddings, context_embeddings

In [396]:
def normalize_data(data):
    row_norms = np.sqrt((data.values**2).sum(axis=1)).reshape(-1,1)
    return data.divide(row_norms, axis='index')

In [397]:
def plot_words(debug):
    plt.figure(figsize=(8,4))
    
    plt.subplot(1,2,1)
    lim_main_first = main_embeddings.loc[[debug[0]]]
    lim_main_second = main_embeddings.loc[[debug[1]]]
    p1 = plt.scatter(lim_main_first[0], lim_main_first[1], color='r')
    plt.arrow(0,0,float(lim_main_first[0]), float(lim_main_first[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_main_first.iterrows():
        plt.text(row[0], row[1], str(idx))
    p2 = plt.scatter(lim_main_second[0], lim_main_second[1], color='r')
    plt.arrow(0,0,float(lim_main_second[0]), float(lim_main_second[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_main_second.iterrows():
        plt.text(row[0], row[1], str(idx))
    sim = 1 - cosine(main_embeddings.loc[debug[0]], main_embeddings.loc[debug[1]])
    plt.title('Sim = %s'%round(sim,4), fontsize=20)
    plt.axvline(0, color='k', linestyle='--', alpha=0.5)
    plt.axhline(0, color='k', linestyle='--', alpha=0.5)
    
    t = np.arange(0, 3.14*2+0.1, 0.1)
    plt.plot(np.cos(t), np.sin(t), linewidth=1, color='k', alpha=0.5, linestyle='--')
    
    ###################################
    plt.subplot(1,2,2)
    lim_main = main_embeddings.loc[[debug[0]]]
    lim_context = context_embeddings.loc[[debug[1]]]
    p1 = plt.scatter(lim_main[0], lim_main[1], color='r')
    plt.arrow(0,0,float(lim_main[0]), float(lim_main[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_main.iterrows():
        plt.text(row[0], row[1], str(idx))
    p2 = plt.scatter(lim_context[0], lim_context[1], color='b')
    plt.arrow(0,0,float(lim_context[0]), float(lim_context[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_context.iterrows():
        plt.text(row[0], row[1], str(idx))
    sim = 1 - cosine(main_embeddings.loc[debug[0]], context_embeddings.loc[debug[1]])
    plt.title('Sim = %s'%round(sim,4), fontsize=20)
    plt.axvline(0, color='k', linestyle='--', alpha=0.5)
    plt.axhline(0, color='k', linestyle='--', alpha=0.5)
    
    plt.plot(np.cos(t), np.sin(t), linewidth=1, color='k', alpha=0.5, linestyle='--')
    
    plt.show()

In [398]:
EMBEDDING_SIZE = 2

main_embeddings = np.random.normal(0,0.1,(len(words), EMBEDDING_SIZE))
row_norms = np.sqrt((main_embeddings**2).sum(axis=1)).reshape(-1,1)
main_embeddings = main_embeddings / row_norms

context_embeddings = np.random.normal(0,0.1,(len(words), EMBEDDING_SIZE))
row_norms = np.sqrt((context_embeddings**2).sum(axis=1)).reshape(-1,1)
context_embeddings = context_embeddings / row_norms

main_embeddings = pd.DataFrame(data=main_embeddings, index=words)
context_embeddings = pd.DataFrame(data=context_embeddings, index=words)

In [399]:
check = []
start = False

#avoid words with numbers followed by measurment units (ie. 100y, 5d, 10cm, etc)
for word in words:
    sim = 1 - cosine(main_embeddings.loc['masculine'], main_embeddings.loc[word])
    if (round(sim, 2)) >= 0.50 or (round(sim, 2)) <= -0.50:
        check.append(word)

for word in words:
    sim = 1 - cosine(main_embeddings.loc['feminine'], main_embeddings.loc[word])
    if (round(sim, 2)) >= 0.50 or (round(sim, 2)) <= -0.50:
        check.append(word)
print(check)
def chat_with_gpt(prompt):
                response = openai.ChatCompletion.create(
                model = "gpt-4-turbo",
                messages = [{"role": "user", "content": prompt}]
                )
                return response.choices[0].message.content.strip()
for word in check:
    if __name__ == "__main__":
        if word != 'masculine' and word != 'feminine':
            response = chat_with_gpt("Is the word " + word + " closer to masculinity, femininity or neutral? Give a one word answer.")
            if response.lower()[0] == "m": 
                f = open('masculine.txt', 'a')
                f.write(word + "\n")
                f.close()
            if response.lower()[0] == "f": 
                f = open('feminine.txt', 'a')
                f.write(word + "\n")
                f.close()

['across', 'acrosstypes', 'aframework', 'allows', 'analysis', 'andengland—obtained', 'apply', 'articles', 'austria', 'automaticallyon', 'beliefs', 'beyondnearby', 'broadly', 'capture', 'captures', 'case', 'close', 'closer', 'collections', 'commonly', 'cooccurrence', 'correspond', 'cultural', 'demonstrate', 'dictionaryand', 'differentlanguages', 'disciplines', 'establishes', 'expensive', 'feminine', 'found', 'gender', 'geometry', 'global', 'highdimensional', 'howeversuch', 'human', 'importanttopic', 'indepth', 'innatural', 'italyand', 'known', 'language', 'languageboth', 'large', 'learning', 'leverage', 'london', 'machine', 'manual', 'many', 'masculine', 'may', 'measure', 'models', 'neighbors', 'nlp', 'often', 'paperwe', 'paris', 'perpetuates', 'playstation', 'primarily', 'processing', 'propose', 'reflects', 'relations', 'relationships', 'relationshipsnot', 'require', 'similar', 'simply', 'specific', 'standardtool', 'stcenturies', 'stereotypes', 'stereotypes–', 'subtracting', 'suchas', 

In [400]:
words = 0
stopwords = []
masculine = 0
feminine = 0
#remove double quotation marks and citations/references if any
with open('training_text.txt') as f1, open('stopwords.txt') as f2:
    for line2 in f2:
        for word in line2.split():
            stopwords.append(word)
    for line1 in f1:
        for word in line1.split():
            if (word) not in stopwords:
                words += 1
                
with open("masculine.txt", 'r') as fp:
    masculine = len(fp.readlines())

with open("feminine.txt", 'r') as fp:
    feminine = len(fp.readlines())

if (masculine >= words * 0.05):
    if (masculine >= (masculine + feminine) * 0.75):
        print("There exists inclusiveness issues.")
    else:
        print("There does not exist inclusiveness issues.")
else:
    print("There does not exist inclusiveness issues.")

There does not exist inclusiveness issues.
